In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

## Making all the constructors functions

div classes

- "col-2 not__cell__767 find" **Rank**
- "col-7 not__cell__767 find" **Name**
- "col-3 not__cell__767 find" **Nation** and **Group**
- "col-5 not__cell__767 find" **Born**
- "col-4 not__cell__767 find" **B. Weight** and **Snatch** and **Cl&Jerk** and **Total**

In [3]:
def find_div(main_html, class_of_div):

    all_results = main_html.findAll("div", class_=class_of_div)

    results_list = []
    for result in all_results:
        results_list.append(result.find('p').text[1:-1]) #this [1:-1] is to eliminate \n n/

    return results_list

In [4]:
def fix_class_problem(main_html, class_of_div, first_letter_hunt):
    results = find_div(main_html, class_of_div)

    results_list = []

    for result in results:
        if result[0] == first_letter_hunt: results_list.append(result)
    
    return results_list

The two functions above will be uses to find the data. The first one is for all the div class that is unique, the second one is for the div classes that are equal. For example, nation and group have the same class, so i use the second function.

In [5]:
def find_html(weight_lifting_url):

    html = requests.get(weight_lifting_url).content
    soup = bs(html, 'html.parser')
    men_html = soup.find("div", id="men_total")
    women_html = soup.find("div", id="women_total")
    event = soup.find("h2")

    htmls = [men_html, women_html, event]
    return htmls

In [6]:
def make_dfs(weight_lifting_url, id):

    htmls = find_html(weight_lifting_url)

    event = htmls[2].text

    men_rank = find_div(htmls[0], "col-2 not__cell__767") 
    men_name = find_div(htmls[0], "col-7 not__cell__767")
    men_born = find_div(htmls[0], "col-5 not__cell__767")
    men_nation = fix_class_problem(htmls[0], "col-3 not__cell__767" , "\n") 
    men_group = fix_class_problem(htmls[0], "col-3 not__cell__767" , "G") 
    men_body_weight = fix_class_problem(htmls[0], "col-4 not__cell__767" , "B") 
    men_snatch = fix_class_problem(htmls[0], "col-4 not__cell__767" , "S")
    men_clean_jerk = fix_class_problem(htmls[0], "col-4 not__cell__767" , "C")
    men_total = fix_class_problem(htmls[0], "col-4 not__cell__767" , "T")

    women_rank = find_div(htmls[1], "col-2 not__cell__767") 
    women_name = find_div(htmls[1], "col-7 not__cell__767")
    women_born = find_div(htmls[1], "col-5 not__cell__767")
    women_nation = fix_class_problem(htmls[1], "col-3 not__cell__767" , "\n") 
    women_group = fix_class_problem(htmls[1], "col-3 not__cell__767" , "G") 
    women_body_weight = fix_class_problem(htmls[1], "col-4 not__cell__767" , "B") 
    women_snatch = fix_class_problem(htmls[1], "col-4 not__cell__767" , "S")
    women_clean_jerk = fix_class_problem(htmls[1], "col-4 not__cell__767" , "C")
    women_total = fix_class_problem(htmls[1], "col-4 not__cell__767" , "T")

    men_dict ={
        "Rank":men_rank,
        "Name":men_name,
        "Nation":men_nation,
        "Born":men_born,
        "B.weight":men_body_weight,
        "Group":men_group,
        "Snatch":men_snatch,
        "CI&Jerk":men_clean_jerk,
        "Total":men_total,
        "Event": event,
        "Event_id":id
    }

    women_dict ={
        "Rank":women_rank,
        "Name":women_name,
        "Nation":women_nation,
        "Born":women_born,
        "B.weight":women_body_weight,
        "Group":women_group,
        "Snatch":women_snatch,
        "CI&Jerk":women_clean_jerk,
        "Total":women_total,
        "Event": event,
        "Event_id":id
    }

    men_df = pd.DataFrame(men_dict)
    women_df = pd.DataFrame(women_dict)

    return [men_df, women_df]

## Setting the urls

In [7]:
new_category_url = f'https://iwf.sport/results/results-by-events/?event_id='
old_category_url = f'https://iwf.sport/results/results-by-events/results-by-events-old-bw/?event_id='

setting the id list

In [8]:
id_df = pd.read_excel("../../data/events_date.xlsx")

In [9]:
def return_id(dataframe):
    new_id=[]
    old_id=[]

    for index in range(len(dataframe)):

        category = dataframe["Category"][index]
        id = dataframe["Event_id"][index]

        if category == "new category": new_id.append(id)
        if category == "old category": old_id.append(id)

    return [new_id, old_id]

In [10]:
ids = return_id(id_df)

In [11]:
new_category_ids = ids[0]
old_category_ids = ids[1]

In [12]:
def create_data_frames(category_url, id_range):

    men_dfs = []
    women_dfs = []
    erros = []

    for id in id_range:

        try:
            dfs = make_dfs(category_url+f'{id}', id)

            men_dfs.append(dfs[0])
            women_dfs.append(dfs[1])

        except:
            erros.append(id)

    men_df = pd.concat(men_dfs)
    women_df = pd.concat(women_dfs)

    return [men_df, women_df, erros]

Creating new category dataframes 

In [13]:
new_category_results = create_data_frames(new_category_url, new_category_ids)

new_category_men_df = new_category_results[0]
new_category_women_df = new_category_results[1]
new_category_erros = new_category_results[2]

In [14]:
new_category_erros

[]

In [15]:
new_category_men_df.head()

,Rank,Name,Nation,Born,B.weight,Group,Snatch,CI&Jerk,Total,Event,Event_id
0,Rank: 1,HIRAI Hayato,\nJPN,"Born: Aug 08, 1994",B.weight: 60.50,Group: A,Snatch: 124,CI&Jerk: 148,Total: 272,5th International Qatar Cup,444
1,Rank: 2,SON Igor,\nKAZ,"Born: Nov 16, 1998",B.weight: 58.60,Group: A,Snatch: 118,CI&Jerk: 149,Total: 267,5th International Qatar Cup,444
2,Rank: 3,OSHIDA Shin,\nJPN,"Born: Jul 29, 1994",B.weight: 60.80,Group: A,Snatch: 115,CI&Jerk: 149,Total: 264,5th International Qatar Cup,444
3,Rank: 4,CHONTEY Arli,\nKAZ,"Born: Jul 01, 1992",B.weight: 57.60,Group: A,Snatch: 115,CI&Jerk: 130,Total: 245,5th International Qatar Cup,444
4,Rank: 1,YOKUBOV Doston,\nUZB,"Born: Apr 05, 1995",B.weight: 67.00,Group: A,Snatch: 138,CI&Jerk: 175,Total: 313,5th International Qatar Cup,444


In [16]:
new_category_women_df.head()

,Rank,Name,Nation,Born,B.weight,Group,Snatch,CI&Jerk,Total,Event,Event_id
0,Rank: 1,ZHANG Rong,\nCHN,"Born: Oct 31, 1993",B.weight: 48.60,Group: A,Snatch: 92,CI&Jerk: 103,Total: 195,5th International Qatar Cup,444
1,Rank: 2,ANDRIES Elena-Ramona,\nROU,"Born: Sep 21, 1994",B.weight: 48.90,Group: A,Snatch: 86,CI&Jerk: 101,Total: 187,5th International Qatar Cup,444
2,Rank: 3,YANAGIDA Mizuki,\nJPN,"Born: Sep 05, 1992",B.weight: 48.90,Group: A,Snatch: 78,CI&Jerk: 98,Total: 176,5th International Qatar Cup,444
3,Rank: 4,MICHEL Anais,\nFRA,"Born: Jan 12, 1988",B.weight: 48.60,Group: A,Snatch: 77,CI&Jerk: 98,Total: 175,5th International Qatar Cup,444
4,Rank: 5,DZHUMABAYEVA Yulduz,\nTKM,"Born: Apr 22, 1998",B.weight: 48.90,Group: A,Snatch: 75,CI&Jerk: 100,Total: 175,5th International Qatar Cup,444


Creating old category dataframes

In [17]:
old_category_results = create_data_frames(old_category_url, old_category_ids)

old_category_men_df = old_category_results[0]
old_category_women_df = old_category_results[1]
old_category_erros = old_category_results[2]

In [18]:
old_category_erros

[107]

In [19]:
old_category_men_df.head()

,Rank,Name,Nation,Born,B.weight,Group,Snatch,CI&Jerk,Total,Event,Event_id
0,Rank: 1,MUTLU Halil,\nTUR,"Born: Jul 14, 1973",B.weight: 55.66,Group: A,Snatch: 135,CI&Jerk: 160,Total: 295,69th MEN's and 12th WOMEN's WORLD CHAMPIONSHIPS,96
1,Rank: 2,LAN Shizhang,\nCHN,"Born: Feb 09, 1974",B.weight: 55.85,Group: A,Snatch: 127.5,CI&Jerk: 157.5,Total: 285,69th MEN's and 12th WOMEN's WORLD CHAMPIONSHIPS,96
2,Rank: 3,IVANOV Ivan,\nBUL,"Born: Jan 01, 1971",B.weight: 55.86,Group: A,Snatch: 122.5,CI&Jerk: 160,Total: 282.5,69th MEN's and 12th WOMEN's WORLD CHAMPIONSHIPS,96
3,Rank: 4,VARGAS William Trujillo,\nCUB,"Born: Sep 17, 1970",B.weight: 55.94,Group: A,Snatch: 127.5,CI&Jerk: 155,Total: 282.5,69th MEN's and 12th WOMEN's WORLD CHAMPIONSHIPS,96
4,Rank: 5,JIGAU Adrian Ioan,\nROU,"Born: Jan 06, 1970",B.weight: 55.79,Group: A,Snatch: 117.5,CI&Jerk: 145,Total: 262.5,69th MEN's and 12th WOMEN's WORLD CHAMPIONSHIPS,96


In [20]:
old_category_women_df.head()

,Rank,Name,Nation,Born,B.weight,Group,Snatch,CI&Jerk,Total,Event,Event_id
0,Rank: 1,LI Yunli,\nCHN,"Born: Jan 01, 1976",B.weight: 47.25,Group: A,Snatch: 80,CI&Jerk: 102.5,Total: 182.5,69th MEN's and 12th WOMEN's WORLD CHAMPIONSHIPS,96
1,Rank: 2,CHU Nan-Mei,\nTPE,"Born: Oct 18, 1970",B.weight: 47.12,Group: A,Snatch: 77.5,CI&Jerk: 95,Total: 172.5,69th MEN's and 12th WOMEN's WORLD CHAMPIONSHIPS,96
2,Rank: 3,TSAI Huey-Woan,\nTPE,"Born: Dec 30, 1970",B.weight: 47.64,Group: A,Snatch: 77.5,CI&Jerk: 95,Total: 172.5,69th MEN's and 12th WOMEN's WORLD CHAMPIONSHIPS,96
3,Rank: 4,IMAOKA Masumi,\nJPN,"Born: Nov 15, 1976",B.weight: 46.29,Group: A,Snatch: 75,CI&Jerk: 92.5,Total: 167.5,69th MEN's and 12th WOMEN's WORLD CHAMPIONSHIPS,96
4,Rank: 5,STOEVA-Toumparova Siyka,\nBUL,"Born: Feb 14, 1973",B.weight: 47.86,Group: A,Snatch: 77.5,CI&Jerk: 90,Total: 167.5,69th MEN's and 12th WOMEN's WORLD CHAMPIONSHIPS,96


## Dataframe construction

In [21]:
def fix_column(column, number_character, is_number):
    new_column = []

    for string in column:

        new_string = string[number_character:]

        if is_number == True:

            try: 
                new_column.append(float(new_string))
            except:
                new_column.append(0)

        else: 
            new_column.append(new_string)
    
    return new_column

In [22]:
def fix_data_frame(dataframe):
    dataframe["Rank"] = fix_column(dataframe["Rank"], 6, True)
    dataframe["Nation"] = fix_column(dataframe["Nation"], 1, False)
    dataframe["Born"] = fix_column(dataframe["Born"], 6, False)
    dataframe["B.weight"] = fix_column(dataframe["B.weight"], 9, True)
    dataframe["Group"] = fix_column(dataframe["Group"], 7, False)
    dataframe["Snatch"] = fix_column(dataframe["Snatch"], 8, True)
    dataframe["CI&Jerk"] = fix_column(dataframe["CI&Jerk"], 9, True)
    dataframe["Total"] = fix_column(dataframe["Total"], 7, True)

    return dataframe

In [23]:
new_category_men_df = fix_data_frame(new_category_men_df)
new_category_women_df = fix_data_frame(new_category_women_df)
old_category_men_df = fix_data_frame(old_category_men_df)
old_category_women_df = fix_data_frame(old_category_women_df)

In [24]:
new_category_men_df.head()

,Rank,Name,Nation,Born,B.weight,Group,Snatch,CI&Jerk,Total,Event,Event_id
0,1.0,HIRAI Hayato,JPN,"Aug 08, 1994",60.5,A,124.0,148.0,272.0,5th International Qatar Cup,444
1,2.0,SON Igor,KAZ,"Nov 16, 1998",58.6,A,118.0,149.0,267.0,5th International Qatar Cup,444
2,3.0,OSHIDA Shin,JPN,"Jul 29, 1994",60.8,A,115.0,149.0,264.0,5th International Qatar Cup,444
3,4.0,CHONTEY Arli,KAZ,"Jul 01, 1992",57.6,A,115.0,130.0,245.0,5th International Qatar Cup,444
4,1.0,YOKUBOV Doston,UZB,"Apr 05, 1995",67.0,A,138.0,175.0,313.0,5th International Qatar Cup,444


Saving as Excel

In [25]:
new_category_men_df.to_excel("../../data/new_category_men_df.xlsx", index=False)
new_category_women_df.to_excel("../../data/new_category_women_df.xlsx", index=False)
old_category_men_df.to_excel("../../data/old_category_men_df.xlsx", index=False)
old_category_women_df.to_excel("../../data/old_category_women_df.xlsx", index=False)